In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import os

import numpy as np

import sys, os
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import frequency
from engaged.features import wave

base_path = os.path.expanduser('~/projects/engaged_hackathon_data/raw_data/cropped_audio/')
dataset_csv = csv.reader(open(base_path + 'below12kHz_cropped_label.txt'))

all_paths = set()    # set of all the original filenames
for single_file_info in dataset_csv:

    wavpath, start_time, wav_class, _ = single_file_info
    original_filename = '_'.join(wavpath.split('_')[:-1])    
    all_paths.add(original_filename)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# discovering which files are not in the other dataset

one_minute_folder = os.path.expanduser(
    '/home/michael/projects/engaged_hackathon_data/raw_data/one_minute_files/25_Random/')

unseen_files = []

for x in os.listdir(one_minute_folder):
    if x[:-4] not in all_paths:
        unseen_files.append(x)

print len(unseen_files)

 13


In [18]:
# now take a subset of these files, and take a subset of the spectrogram for each one
from skimage.util.shape import view_as_blocks

# num_files = 10
snippet_length_seconds = 30

slice_width_in_seconds = 0.5

spectrogram_parameters = {
    'nfft': 1024,
    'window_width': 0.03,
    'overlap': 0.01
    }

import random

files_to_use = unseen_files
# random.sample(unseen_files, num_files)

all_slices = []

for filepath in files_to_use:
    
    # load file and cut down to size
    wav = wave.Wave()
    wav.load_from_wav(one_minute_folder + filepath)
    wav = wav.snippet(None, snippet_length_seconds)
    
    # compute spec
    spec = wave.Spectrogram()
    spec.from_wave(wav, **spectrogram_parameters)
    spec.spec -= np.median(spec.spec, axis=1)[:, None]
    
    slice_width = int(slice_width_in_seconds / spec.window_width)
    block_shape = (spec.nfft/2, slice_width)
    
    # convert to slices
    max_width = np.floor(spec.spec.shape[1] / slice_width) * slice_width
    spec.spec = np.ascontiguousarray(spec.spec[:, :max_width])

    view = view_as_blocks(spec.spec, block_shape)
    view = np.reshape(view, (-1, block_shape[0] * block_shape[1]))
    
    all_slices.append(view)

0


ValueError: negative dimensions are not allowed

In [5]:
from sklearn.cluster import MiniBatchKMeans
all_slices_np = np.vstack(all_slices)

km = MiniBatchKMeans(n_clusters=200)
km.fit(all_slices_np)


MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=200, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)

In [6]:
# saving the dictionary
import cPickle as pickle
obj = {'km':km, 'slice_width_in_seconds':slice_width_in_seconds}

savepath = '/home/michael/projects/engaged_hackathon_data/dictionaries/200_words.pkl'
pickle.dump(obj, open(savepath, 'w'))

In [7]:
print all_slices_np.shape

(18700, 8192)


In [8]:
# doing pca on this
from sklearn.decomposition import RandomizedPCA
pca = RandomizedPCA(n_components=100)
pca.fit(all_slices_np)

RandomizedPCA(copy=True, iterated_power=3, n_components=100,
       random_state=None, whiten=False)

In [9]:
reconstituted = pca.inverse_transform(pca.transform(all_slices_np))

for count, cen in enumerate(reconstituted[:30]):
    plt.subplot(1, maxi, count+1)    
    plt.imshow(cen.reshape((512, slice_width)))
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.clim(-2.5, 2.5)
    

NameError: name 'maxi' is not defined

In [ ]:
# visualise the cluster centers
plt.figure(figsize=(15, 15))
maxi = 30

for count, cen in enumerate(km.cluster_centers_[30:30+maxi]):
    plt.subplot(1, maxi, count+1)    
    plt.imshow(cen.reshape((512, slice_width)))
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.clim(-2.5, 2.5)

In [ ]:
plt.figure(figsize=(15, 15))
mean_clust = km.cluster_centers_.mean(0)
clust_dists = np.sum((km.cluster_centers_ - mean_clust)**2, axis=1)
sorted_clust_idxs = np.argsort(clust_dists)

plt.plot(clust_dists[sorted_clust_idxs])


for count, idx in enumerate(sorted_clust_idxs[:maxi]):
    plt.subplot(1, maxi, count+1)    
    plt.imshow(km.cluster_centers_[idx].reshape((512, slice_width)))
    plt.axis('off')
    plt.gca().invert_yaxis()
    plt.clim(-2.5, 2.5)

In [ ]:
# now look at freuqncy
# zip(np.bincount(km.labels_), sorted_clust_idxs)
np.bincount(km.labels_)[sorted_clust_idxs]
